In [5]:
import pandas as pd
import numpy as np
import tpqoa
from datetime import datetime, timezone, timedelta 
import time
import warnings
warnings.filterwarnings('ignore')

In [18]:
class ConTrader(tpqoa.tpqoa):
    def __init__(self, conf_file, instrument, bar_length, window, units):
        super().__init__(conf_file)
        self.instrument = instrument
        self.bar_length = pd.to_timedelta(bar_length)
        self.tick_data = pd.DataFrame()
        self.raw_data = None
        self.data = None 
        self.last_bar = None
        self.units = units
        self.position = 0
        self.profits = []
        
        #*****************add strategy-specific attributes here******************
        self.window = window
        #************************************************************************
    
    def get_most_recent(self, days = 5):
        while True:
            time.sleep(2)
            now = datetime.now(timezone.utc).replace(tzinfo=None)
            now = now - timedelta(microseconds = now.microsecond)
            past = now - timedelta(days = days)
            df = self.get_history(instrument = self.instrument, start = past, end = now,
                                   granularity = "S5", price = "M", localize = False).c.dropna().to_frame()
            df.rename(columns = {"c":self.instrument}, inplace = True)
            df = df.resample(self.bar_length, label = "right").last().dropna().iloc[:-1]
            self.raw_data = df.copy()
            self.last_bar = self.raw_data.index[-1]
            if pd.to_datetime(datetime.now(timezone.utc)) - self.last_bar < self.bar_length:
                break
            
    def on_success(self, time, bid, ask):
        print(self.ticks, end = " ")
        
    
        recent_tick = pd.to_datetime(time)
        df = pd.DataFrame({self.instrument:(ask + bid)/2}, 
                          index = [recent_tick])
        self.tick_data = pd.concat([self.tick_data, df]) 
        
    
        if recent_tick - self.last_bar >= self.bar_length:
            self.resample_and_join()
            self.define_strategy()
            self.execute_trades()
            
    def resample_and_join(self):
        self.raw_data = pd.concat([self.raw_data, self.tick_data.resample(self.bar_length, 
                                                                          label="right").last().ffill().iloc[:-1]]) 
        self.tick_data = self.tick_data.iloc[-1:]
        self.last_bar = self.raw_data.index[-1]
        
    def define_strategy(self): 
        df = self.raw_data.copy()
        
        #******************** define your strategy here ************************
        df["returns"] = np.log(df[self.instrument] / df[self.instrument].shift())
        df["position"] = -np.sign(df.returns.rolling(self.window).mean())
        #***********************************************************************
        
        self.data = df.copy()
        
    def execute_trades(self):
        if self.data["position"].iloc[-1] == 1:
            if self.position == 0:
                order = self.create_order(self.instrument, self.units, suppress = True, ret = True)
                self.report_trade(order, "GOING LONG")
            elif self.position == -1:
                order = self.create_order(self.instrument, self.units * 2, suppress = True, ret = True) 
                self.report_trade(order, "GOING LONG")
            self.position = 1
        elif self.data["position"].iloc[-1] == -1: 
            if self.position == 0:
                order = self.create_order(self.instrument, -self.units, suppress = True, ret = True)
                self.report_trade(order, "GOING SHORT")
            elif self.position == 1:
                order = self.create_order(self.instrument, -self.units * 2, suppress = True, ret = True)
                self.report_trade(order, "GOING SHORT")
            self.position = -1
        elif self.data["position"].iloc[-1] == 0: 
            if self.position == -1:
                order = self.create_order(self.instrument, self.units, suppress = True, ret = True) 
                self.report_trade(order, "GOING NEUTRAL")
            elif self.position == 1:
                order = self.create_order(self.instrument, -self.units, suppress = True, ret = True)
                self.report_trade(order, "GOING NEUTRAL")
            self.position = 0
    
    def report_trade(self, order, going): 
        time = order["time"]
        units = order["units"]
        price = order["price"]
        pl = float(order["pl"])
        self.profits.append(pl)
        cumpl = sum(self.profits)
        print("\n" + 100* "-")
        print("{} | {}".format(time, going))
        print("{} | units = {} | price = {} | P&L = {} | Cum P&L = {}".format(time, units, price, pl, cumpl))
        print(100 * "-" + "\n")  
  

In [19]:
trader = ConTrader("/Users/user/Desktop/Oanda_firststeps/oanda.cfg", "EUR_USD", "1m", window = 3, units = 100)



KeyboardInterrupt



In [20]:
trader.get_most_recent()
trader.stream_data(trader.instrument, stop = 200)
if trader.position != 0: # if we have a final open position
    close_order = trader.create_order(trader.instrument, units = -trader.position * trader.units, 
                                      suppress = True, ret = True) 
    trader.report_trade(close_order, "GOING NEUTRAL")
    trader.position = 0

KeyboardInterrupt: 

In [69]:
trader.get_position()

0

In [62]:
trader.profits

100

In [66]:
sum(trader.profits)

[{'instrument': 'EUR_USD',
  'pl': '1081.994',
  'unrealizedPL': '-0.011',
  'marginUsed': '3.9249',
  'resettablePL': '1081.994',
  'financing': '-54.8633',
  'commission': '0.0',
  'guaranteedExecutionFees': '0.0',
  'long': {'units': '100.0',
   'averagePrice': 1.17754,
   'tradeIDs': ['66'],
   'pl': '1081.994',
   'unrealizedPL': '-0.011',
   'resettablePL': '1081.994',
   'financing': '-54.8633',
   'guaranteedExecutionFees': '0.0'},
  'short': {'units': '0.0',
   'pl': '0.0',
   'unrealizedPL': '0.0',
   'resettablePL': '0.0',
   'financing': '0.0',
   'guaranteedExecutionFees': '0.0'}}]

In [68]:
if trader.position != 0:
    order = trader.create_order(trader.instrument, units = -trader.position * trader.units, suppress = True, ret = True)
    trader.position = 0

In [70]:
trader.data

,EUR_USD,returns,position
2025-09-14 21:05:00+00:00,1.173310,NaN,NaN
2025-09-14 21:06:00+00:00,1.173320,0.000009,NaN
2025-09-14 21:07:00+00:00,1.173300,-0.000017,NaN
2025-09-14 21:08:00+00:00,1.173300,0.000000,1.0
2025-09-14 21:09:00+00:00,1.173300,0.000000,1.0
...,...,...,...
2025-09-18 16:29:00+00:00,1.177520,-0.000034,1.0
2025-09-18 16:30:00+00:00,1.177620,0.000085,-1.0
2025-09-18 16:31:00+00:00,1.177550,-0.000059,1.0
2025-09-18 16:32:00+00:00,1.177400,-0.000127,1.0


In [13]:
df = api.get_history(instrument = instrument, start = yesterday, end = now,
                     granularity = "S5", price = "M", localize = False)
df

,o,h,l,c,volume,complete
time,,,,,,
2025-09-17 12:48:10+00:00,1.18418,1.18420,1.18417,1.18418,9,True
2025-09-17 12:48:20+00:00,1.18420,1.18420,1.18418,1.18420,3,True
2025-09-17 12:48:25+00:00,1.18420,1.18428,1.18419,1.18428,27,True
2025-09-17 12:48:30+00:00,1.18427,1.18428,1.18426,1.18427,13,True
2025-09-17 12:48:35+00:00,1.18428,1.18428,1.18426,1.18428,5,True
...,...,...,...,...,...,...
2025-09-18 12:47:55+00:00,1.18068,1.18068,1.18058,1.18062,34,True
2025-09-18 12:48:00+00:00,1.18063,1.18064,1.18051,1.18054,40,True
2025-09-18 12:48:05+00:00,1.18054,1.18054,1.18046,1.18048,44,True


In [15]:
df = df.c.dropna().to_frame()
df

,c
time,
2025-09-17 12:48:10+00:00,1.18418
2025-09-17 12:48:20+00:00,1.18420
2025-09-17 12:48:25+00:00,1.18428
2025-09-17 12:48:30+00:00,1.18427
2025-09-17 12:48:35+00:00,1.18428
...,...
2025-09-18 12:47:55+00:00,1.18062
2025-09-18 12:48:00+00:00,1.18054
2025-09-18 12:48:05+00:00,1.18048


In [18]:
df.rename(columns = {"c":instrument}, inplace = True)
df

,EUR_USD
time,
2025-09-17 12:48:10+00:00,1.18418
2025-09-17 12:48:20+00:00,1.18420
2025-09-17 12:48:25+00:00,1.18428
2025-09-17 12:48:30+00:00,1.18427
2025-09-17 12:48:35+00:00,1.18428
...,...
2025-09-18 12:47:55+00:00,1.18062
2025-09-18 12:48:00+00:00,1.18054
2025-09-18 12:48:05+00:00,1.18048


In [19]:
df = df.resample("1min", label = "right").last().dropna().iloc[:-1]
df

,EUR_USD
time,
2025-09-17 12:49:00+00:00,1.18444
2025-09-17 12:50:00+00:00,1.18424
2025-09-17 12:51:00+00:00,1.18420
2025-09-17 12:52:00+00:00,1.18409
2025-09-17 12:53:00+00:00,1.18400
...,...
2025-09-18 12:44:00+00:00,1.18026
2025-09-18 12:45:00+00:00,1.18060
2025-09-18 12:46:00+00:00,1.18045


In [20]:
last_bar = df.index[-1]
last_bar

Timestamp('2025-09-18 12:48:00+0000', tz='UTC')